# Easy Scalping Strategy Part III - The Candle Wick Condition

In [1]:
import pandas as pd
df = pd.read_csv("EURUSD_Candlestick_15_M_BID_31.01.2019-30.01.2022.csv")
df.tail()

,Local time,Open,High,Low,Close,Volume
105123,30.01.2022 00:45:00.000 GMT+0200,1.1145,1.1145,1.1145,1.1145,0.0
105124,30.01.2022 01:00:00.000 GMT+0200,1.1145,1.1145,1.1145,1.1145,0.0
105125,30.01.2022 01:15:00.000 GMT+0200,1.1145,1.1145,1.1145,1.1145,0.0
105126,30.01.2022 01:30:00.000 GMT+0200,1.1145,1.1145,1.1145,1.1145,0.0
105127,30.01.2022 01:45:00.000 GMT+0200,1.1145,1.1145,1.1145,1.1145,0.0


In [2]:
df=df[df['Volume']!=0]
df.isna().sum()
df.reset_index(drop=True, inplace=True)
df.tail()

,Local time,Open,High,Low,Close,Volume
74758,28.01.2022 22:45:00.000 GMT+0200,1.11448,1.11503,1.11438,1.11502,457.00
74759,28.01.2022 23:00:00.000 GMT+0200,1.11502,1.11505,1.11457,1.11474,143.91
74760,28.01.2022 23:15:00.000 GMT+0200,1.11475,1.11485,1.11460,1.11470,171.68
74761,28.01.2022 23:30:00.000 GMT+0200,1.11469,1.11513,1.11469,1.11505,228.56
74762,28.01.2022 23:45:00.000 GMT+0200,1.11505,1.11506,1.11449,1.11450,1568.78


In [3]:
import pandas_ta as ta
df["EMA50"] = ta.ema(df.Close, length=50)
df["EMA100"] = ta.ema(df.Close, length=100)
df["EMA150"] = ta.ema(df.Close, length=150)
df['ATR']= df.ta.atr()

In [4]:
backrollingN = 10
df['slopeEMA50'] = df['EMA50'].diff(periods=1)
df['slopeEMA50'] = df['slopeEMA50'].rolling(window=backrollingN).mean()

df['slopeEMA100'] = df['EMA100'].diff(periods=1)
df['slopeEMA100'] = df['slopeEMA100'].rolling(window=backrollingN).mean()

df['slopeEMA150'] = df['EMA150'].diff(periods=1)
df['slopeEMA150'] = df['slopeEMA150'].rolling(window=backrollingN).mean()

In [5]:
df.tail()

,Local time,Open,High,Low,Close,Volume,EMA50,EMA100,EMA150,ATR,slopeEMA50,slopeEMA100,slopeEMA150
74758,28.01.2022 22:45:00.000 GMT+0200,1.11448,1.11503,1.11438,1.11502,457.00,1.114836,1.115303,1.116403,0.000728,-0.000005,-0.000012,-0.000024
74759,28.01.2022 23:00:00.000 GMT+0200,1.11502,1.11505,1.11457,1.11474,143.91,1.114832,1.115291,1.116381,0.000711,-0.000007,-0.000013,-0.000024
74760,28.01.2022 23:15:00.000 GMT+0200,1.11475,1.11485,1.11460,1.11470,171.68,1.114827,1.115280,1.116358,0.000678,-0.000009,-0.000014,-0.000024
74761,28.01.2022 23:30:00.000 GMT+0200,1.11469,1.11513,1.11469,1.11505,228.56,1.114835,1.115275,1.116341,0.000661,-0.000007,-0.000013,-0.000024
74762,28.01.2022 23:45:00.000 GMT+0200,1.11505,1.11506,1.11449,1.11450,1568.78,1.114822,1.115260,1.116317,0.000654,-0.000007,-0.000013,-0.000023


## EMA Signal

In [6]:
import numpy as np
#conditions = [
#    ( (df['EMA50']<df['EMA100']) & (df['EMA100']<df['EMA150']) & (df['slopeEMA50']<0) & (df['slopeEMA100']<0) & (df['slopeEMA150']<0) ),
#    ( (df['EMA50']>df['EMA100']) & (df['EMA100']>df['EMA150']) & (df['slopeEMA50']>0) & (df['slopeEMA100']>0) & (df['slopeEMA150']>0) )
#           ]
conditions = [
    ( (df['EMA50']<df['EMA100']) & (df['EMA100']<df['EMA150']) & (df['slopeEMA50']<0) & (df['slopeEMA100']<0) & (df['slopeEMA150']<0)),
    ( (df['EMA50']>df['EMA100']) & (df['EMA100']>df['EMA150']) & (df['slopeEMA50']>0) & (df['slopeEMA100']>0) & (df['slopeEMA150']>0))
           ]
choices = [1, 2]
df['EMAsignal'] = np.select(conditions, choices, default=0)

In [7]:
df.tail()

,Local time,Open,High,Low,Close,Volume,EMA50,EMA100,EMA150,ATR,slopeEMA50,slopeEMA100,slopeEMA150,EMAsignal
74758,28.01.2022 22:45:00.000 GMT+0200,1.11448,1.11503,1.11438,1.11502,457.00,1.114836,1.115303,1.116403,0.000728,-0.000005,-0.000012,-0.000024,1
74759,28.01.2022 23:00:00.000 GMT+0200,1.11502,1.11505,1.11457,1.11474,143.91,1.114832,1.115291,1.116381,0.000711,-0.000007,-0.000013,-0.000024,1
74760,28.01.2022 23:15:00.000 GMT+0200,1.11475,1.11485,1.11460,1.11470,171.68,1.114827,1.115280,1.116358,0.000678,-0.000009,-0.000014,-0.000024,1
74761,28.01.2022 23:30:00.000 GMT+0200,1.11469,1.11513,1.11469,1.11505,228.56,1.114835,1.115275,1.116341,0.000661,-0.000007,-0.000013,-0.000024,1
74762,28.01.2022 23:45:00.000 GMT+0200,1.11505,1.11506,1.11449,1.11450,1568.78,1.114822,1.115260,1.116317,0.000654,-0.000007,-0.000013,-0.000023,1


## Total signal

In [8]:
TotSignal = [0] * len(df)
wicklimit=2e-5
for row in range(0, len(df)):
    TotSignal[row] = 0
    if df.EMAsignal[row]==1 and df.Open[row]>df.EMA50[row] and df.Close[row]<df.EMA50[row] and df.High[row]-df.Open[row]<=wicklimit:
        TotSignal[row]=1
    if df.EMAsignal[row]==2 and df.Open[row]<df.EMA50[row] and df.Close[row]>df.EMA50[row] and df.Open[row]-df.Low[row]<=wicklimit:
        TotSignal[row]=2

df['TotSignal']=TotSignal

In [9]:
df.tail()

,Local time,Open,High,Low,Close,Volume,EMA50,EMA100,EMA150,ATR,slopeEMA50,slopeEMA100,slopeEMA150,EMAsignal,TotSignal
74758,28.01.2022 22:45:00.000 GMT+0200,1.11448,1.11503,1.11438,1.11502,457.00,1.114836,1.115303,1.116403,0.000728,-0.000005,-0.000012,-0.000024,1,0
74759,28.01.2022 23:00:00.000 GMT+0200,1.11502,1.11505,1.11457,1.11474,143.91,1.114832,1.115291,1.116381,0.000711,-0.000007,-0.000013,-0.000024,1,0
74760,28.01.2022 23:15:00.000 GMT+0200,1.11475,1.11485,1.11460,1.11470,171.68,1.114827,1.115280,1.116358,0.000678,-0.000009,-0.000014,-0.000024,1,0
74761,28.01.2022 23:30:00.000 GMT+0200,1.11469,1.11513,1.11469,1.11505,228.56,1.114835,1.115275,1.116341,0.000661,-0.000007,-0.000013,-0.000024,1,0
74762,28.01.2022 23:45:00.000 GMT+0200,1.11505,1.11506,1.11449,1.11450,1568.78,1.114822,1.115260,1.116317,0.000654,-0.000007,-0.000013,-0.000023,1,1


In [10]:
#df.dropna(inplace=True)
#df.reset_index(drop=True, inplace=True)

## Visualizing signals on chart

In [11]:
def pointpos(x):
    if x['TotSignal']==1:
        return x['High']+1e-3
    elif x['TotSignal']==2:
        return x['Low']-1e-3
    else:
        return np.nan

df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)

In [137]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime

months = 24*4*30
startid = 67000
dfpl = df[startid:startid+1*months]

fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close']),
                go.Scatter(x=dfpl.index, y=dfpl.EMA50, line=dict(color='orange', width=1), name="EMA50"),
                go.Scatter(x=dfpl.index, y=dfpl.EMA100, line=dict(color='blue', width=1), name="EMA100"),
                go.Scatter(x=dfpl.index, y=dfpl.EMA150, line=dict(color='red', width=1), name="EMA150")])

fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="Signal")
fig.show()

In [69]:
dfpl[1900:1905]

,Local time,Open,High,Low,Close,Volume,EMA50,EMA100,EMA150,ATR,slopeEMA50,slopeEMA100,slopeEMA150,EMAsignal,TotSignal,pointpos
68900,04.11.2021 21:15:00.000 GMT+0200,1.15520,1.15555,1.15516,1.15552,497.80,1.155664,1.156703,1.157279,0.000593,-0.000027,-0.000035,-0.000031,1,0,NaN
68901,04.11.2021 21:30:00.000 GMT+0200,1.15550,1.15565,1.15537,1.15561,727.71,1.155662,1.156682,1.157257,0.000571,-0.000022,-0.000033,-0.000030,1,0,NaN
68902,04.11.2021 21:45:00.000 GMT+0200,1.15563,1.15574,1.15547,1.15573,323.55,1.155664,1.156663,1.157237,0.000549,-0.000018,-0.000030,-0.000028,1,0,NaN
68903,04.11.2021 22:00:00.000 GMT+0200,1.15571,1.15578,1.15549,1.15555,329.56,1.155660,1.156641,1.157214,0.000531,-0.000014,-0.000028,-0.000026,1,0,NaN
68904,04.11.2021 22:15:00.000 GMT+0200,1.15555,1.15566,1.15549,1.15556,148.40,1.155656,1.156619,1.157193,0.000505,-0.000010,-0.000026,-0.000025,1,0,NaN


## Backtesting Strategy

In [12]:
months = 24*4*30
startid = 0
dfpl = df[startid:startid+30*months]

def SIGNAL():
    return dfpl.TotSignal

In [13]:
from backtesting import Strategy
class MyStrat(Strategy):
    initsize = 0.3
    mysize = initsize
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next()
        
        #if(self.signal1>0 and len(self.trades)==0 and len(self.closed_trades)>0 and self.closed_trades[-1].pl < 0):
        #    self.mysize=self.mysize*2
        #elif len(self.closed_trades)>0 and self.closed_trades[-1].pl > 0:
        #    self.mysize=self.initsize

        if self.signal1==2 and len(self.trades)==0:   
            sl1 = self.data.Close[-1] - 45e-4
            tp1 = self.data.Close[-1] + 45e-4 #45
            self.buy(sl=sl1, tp=tp1, size=self.mysize)
        
        elif self.signal1==1 and len(self.trades)==0:         
            sl1 = self.data.Close[-1] + 45e-4
            tp1 = self.data.Close[-1] - 45e-4 #45
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

D:\ProgramFiles\Anaconda3\lib\site-packages\backtesting\_plotting.py:47: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [14]:
from backtesting import Backtest

bt = Backtest(dfpl, MyStrat, cash=100, margin=1/100, commission=.00)
stat = bt.run()
stat

D:\ProgramFiles\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  This is separate from the ipykernel package so we can avoid doing imports until


Start                                     0.0
End                                   74762.0
Duration                              74762.0
Exposure Time [%]                   39.241336
Equity Final [$]                   1905.80291
Equity Peak [$]                    2425.52537
Return [%]                         1805.80291
Buy & Hold Return [%]               -2.942636
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -63.855703
Avg. Drawdown [%]                   -4.369021
Max. Drawdown Duration                21484.0
Avg. Drawdown Duration             324.794258
# Trades                                196.0
Win Rate [%]                        59.183673
Best Trade [%]                       0.442298
Worst Trade [%]                     -0.416659
Avg. Trade [%]                    

In [15]:
#bt.plot(show_legend=False)

In [142]:
#ATR related SL TP
months = 24*4*30
startid = 0
dfpl = df[startid:startid+30*months]
from backtesting import Strategy
class MyStrat(Strategy):
    initsize = 0.3
    mysize = initsize
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next()
        slatr = 2*self.data.ATR[-1]
        TPSLRatio = 1.2
        #if(self.signal1>0 and len(self.trades)==0 and len(self.closed_trades)>0 and self.closed_trades[-1].pl < 0):
        #    self.mysize=self.mysize*2
        #elif len(self.closed_trades)>0 and self.closed_trades[-1].pl > 0:
        #    self.mysize=self.initsize

        if self.signal1==2 and len(self.trades)==0:   
            sl1 = self.data.Close[-1] - slatr
            tp1 = self.data.Close[-1] + slatr*TPSLRatio
            self.buy(sl=sl1, tp=tp1, size=self.mysize)
        
        elif self.signal1==1 and len(self.trades)==0:         
            sl1 = self.data.Close[-1] + slatr
            tp1 = self.data.Close[-1] - slatr*TPSLRatio
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

bt = Backtest(dfpl, MyStrat, cash=100, margin=1/100, commission=.000)
stat = bt.run()
stat

D:\ProgramFiles\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: UserWarning:

Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.



Start                                     0.0
End                                   74762.0
Duration                              74762.0
Exposure Time [%]                    6.931236
Equity Final [$]                   349.482255
Equity Peak [$]                    398.165586
Return [%]                         249.482255
Buy & Hold Return [%]               -2.942636
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -26.423724
Avg. Drawdown [%]                   -2.933579
Max. Drawdown Duration                19334.0
Avg. Drawdown Duration             617.676471
# Trades                                319.0
Win Rate [%]                        51.724138
Best Trade [%]                       0.428416
Worst Trade [%]                     -0.296694
Avg. Trade [%]                    

In [16]:
#bt.plot(show_legend=False)

In [144]:
#Trailing SL with ATR related distance
months = 24*4*30
startid = 0
dfpl = df[startid:startid+30*months]
from backtesting import Strategy, Backtest
import numpy as np

class MyStrat(Strategy):
    
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next()
        sltr = 1*self.data.ATR[-1]
        for trade in self.trades: 
            if trade.is_long: 
                trade.sl = max(trade.sl or -np.inf, self.data.Close[-1] - sltr)
                if self.signal1==1:
                    trade.close()
            else:
                trade.sl = min(trade.sl or np.inf, self.data.Close[-1] + sltr) 
                if self.signal1==2:
                    trade.close()
                    
        if self.signal1==2 and len(self.trades)==0:
            sl1 = self.data.Close[-1] - sltr
            self.buy(sl=sl1)
        elif self.signal1==1 and len(self.trades)==0:
            sl1 = self.data.Close[-1] + sltr
            self.sell(sl=sl1)


bt = Backtest(dfpl, MyStrat, cash=100, margin=1/100, commission=.000)
stat = bt.run()
stat

D:\ProgramFiles\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: UserWarning:

Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.



Start                                     0.0
End                                   74762.0
Duration                              74762.0
Exposure Time [%]                    3.012185
Equity Final [$]                   494.827142
Equity Peak [$]                   1050.069895
Return [%]                         394.827142
Buy & Hold Return [%]               -2.942636
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -55.091838
Avg. Drawdown [%]                  -10.308686
Max. Drawdown Duration                26303.0
Avg. Drawdown Duration            1349.163636
# Trades                                369.0
Win Rate [%]                        42.547425
Best Trade [%]                       0.554613
Worst Trade [%]                     -0.147885
Avg. Trade [%]                    

In [17]:
#bt.plot()

# Stop Loss from Price

In [146]:
SLSignal = [0] * len(df)
SLbackcandles = 6
for row in range(SLbackcandles, len(df)):
    mi=1e10
    ma=-1e10
    if df.TotSignal[row]==1:
        for i in range(row-SLbackcandles, row+1):
            ma = max(ma,df.High[i])
        SLSignal[row]=ma
    if df.TotSignal[row]==2:
        for i in range(row-SLbackcandles, row+1):
            mi = min(mi,df.Low[i])
        SLSignal[row]=mi
        
df['SLSignal']=SLSignal

In [147]:
df

,Local time,Open,High,Low,Close,Volume,EMA50,EMA100,EMA150,ATR,slopeEMA50,slopeEMA100,slopeEMA150,EMAsignal,TotSignal,pointpos,SLSignal
0,31.01.2019 00:00:00.000 GMT+0200,1.14786,1.14857,1.14782,1.14829,1170.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,0.00000
1,31.01.2019 00:15:00.000 GMT+0200,1.14829,1.14858,1.14823,1.14849,1842.58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,0.00000
2,31.01.2019 00:30:00.000 GMT+0200,1.14853,1.14854,1.14813,1.14819,1002.61,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,0.00000
3,31.01.2019 00:45:00.000 GMT+0200,1.14818,1.14828,1.14804,1.14805,609.26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,0.00000
4,31.01.2019 01:00:00.000 GMT+0200,1.14809,1.14850,1.14809,1.14849,1669.71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74758,28.01.2022 22:45:00.000 GMT+0200,1.11448,1.11503,1.11438,1.11502,457.00,1.114836,1.115303,1.116403,0.000728,-0.000005,-0.000012,-0.000024,1,0,NaN,0.00000
74759,28.01.2022 23:00:00.000 GMT+0200,1.11502,1.11505,1.11457,1.11474,143.91,1.114832,1.115291,1.116381,0.000711,-0.000007,-0.000013,-0.000024,1,0,NaN,0.00000
74760,28.01.2022 23:15:00.000 GMT+0200,1.11475,1.11485,1.11460,1.11470,171.68,1.114827,1.115280,1.116358,0.000678,-0.000009,-0.000014,-0.000024,1,0,NaN,0.00000
74761,28.01.2022 23:30:00.000 GMT+0200,1.11469,1.11513,1.11469,1.11505,228.56,1.114835,1.115275,1.116341,0.000661,-0.000007,-0.000013,-0.000024,1,0,NaN,0.00000


In [148]:
dfpl = df[:]
from backtesting import Strategy
from backtesting import Backtest

class MyStrat(Strategy):
    initsize = 0.999
    mysize = initsize
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next()
        TPSLRatio = 1.2

        if self.signal1==2 and len(self.trades)==0:   
            sl1 = self.data.SLSignal[-1]
            tp1 = self.data.Close[-1]+(self.data.Close[-1] - sl1)*TPSLRatio
            self.buy(sl=sl1, tp=tp1, size=self.mysize)
        
        elif self.signal1==1 and len(self.trades)==0:         
            sl1 = self.data.SLSignal[-1]
            tp1 = self.data.Close[-1]-(sl1 - self.data.Close[-1])*TPSLRatio
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

bt = Backtest(dfpl, MyStrat, cash=100, margin=1/100, commission=.00)
stat = bt.run()
stat

D:\ProgramFiles\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: UserWarning:

Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.



Start                                     0.0
End                                   74762.0
Duration                              74762.0
Exposure Time [%]                     4.39121
Equity Final [$]                   473.625372
Equity Peak [$]                    619.737502
Return [%]                         373.625372
Buy & Hold Return [%]               -2.942636
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -74.819392
Avg. Drawdown [%]                  -10.178161
Max. Drawdown Duration                27636.0
Avg. Drawdown Duration                 1712.3
# Trades                                351.0
Win Rate [%]                        49.002849
Best Trade [%]                        0.32277
Worst Trade [%]                     -0.382591
Avg. Trade [%]                    

In [18]:
#bt.plot()